In [1]:
import pandas as pd

In [22]:
contributions = pd.read_csv('individual_contributions.csv', sep='|', header=None,
                            usecols=[0, 3, 10, 13, 14, 20],
                            names=['CMTE_ID',
                                  'TRANSACTION_PGI',
                                  'ZIP_CODE',
                                  'TRANSACTION_DT',
                                  'TRANSACTION_AMT',
                                  'SUB_ID'],
                            dtype={"CMTE_ID": str, 
                                   "TRANSACTION_PGI": str,
                                   "ZIP_CODE": str,
                                  'TRANSACTION_DT': str,
                                   "TRANSACTION_AMT": float, 
                                   "SUB_ID": str
                                  }
                           )

In [23]:
contributions['zipcode_5'] = contributions['ZIP_CODE'].str[0:5]
contributions['ZIP_CODE'] = pd.to_numeric(contributions['ZIP_CODE'], errors='coerce')

In [24]:
contributions.dtypes

CMTE_ID             object
TRANSACTION_PGI     object
ZIP_CODE           float64
TRANSACTION_DT      object
TRANSACTION_AMT    float64
SUB_ID              object
zipcode_5           object
dtype: object

In [25]:
contributions.dropna(subset=['CMTE_ID', 'ZIP_CODE','TRANSACTION_AMT','SUB_ID','zipcode_5'],inplace=True)
contributions = contributions.loc[contributions['TRANSACTION_AMT'] > 0]

In [26]:
contributions.head()

,CMTE_ID,TRANSACTION_PGI,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,SUB_ID,zipcode_5
0,C00088591,P,220424511.0,02132015,500.0,4032020151240885624,22042
1,C00088591,P,220424511.0,02132015,200.0,4032020151240885819,22042
2,C00088591,P,220424511.0,02272015,200.0,4032020151240885820,22042
3,C00088591,P,220424511.0,02132015,200.0,4032020151240885683,22042
4,C00088591,P,220424511.0,02272015,200.0,4032020151240885684,22042


In [6]:
contributions.to_csv('contributions.csv', index=False)

In [27]:
grouped = contributions.groupby(['zipcode_5'])

In [28]:
zip_summary_df = pd.DataFrame()
zip_summary_df['donations_sum'] = grouped['TRANSACTION_AMT'].sum()
zip_summary_df['donations_median'] = grouped['TRANSACTION_AMT'].median()
zip_summary_df['donations_count'] = grouped['SUB_ID'].count()
zip_summary_df.reset_index(inplace=True)
zip_summary_df['zipcode_5'] = zip_summary_df['zipcode_5'].astype(str)

In [29]:
zip_summary_df.head()

,zipcode_5,donations_sum,donations_median,donations_count
0,\t6410,798.0,42.0,19
1,\t8400,15.0,15.0,1
2,3102,400.0,100.0,4
3,6410,4388.0,100.0,32
4,8411,1020.0,42.5,24


In [30]:
zip_summary_df.dtypes

zipcode_5            object
donations_sum       float64
donations_median    float64
donations_count       int64
dtype: object

In [31]:
zip_summary_df.to_csv('zipcode_donations.csv', index=False)

In [32]:
census_df = pd.read_csv('census_data_2016.csv')
#https://www.census.gov/glossary/#term_Employed

C:\Users\alexa\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,76,78,80,82,84,86,87,88,89,90,91,92,93,94,95,96,97,101,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,150,151,152,153,154,155,156,157,158,159,160,161,162,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
census_df.columns = census_df.iloc[0]
census_df.drop(0, axis=0, inplace=True)
census_df.head()

,Id,Id2,Geography,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,...,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Less than high school graduate,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - High school graduate (includes equivalency),Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Some college or associate's degree,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Bachelor's degree or higher,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Less than high school graduate,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - High school graduate (includes equivalency),Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Some college or associate's degree,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Bachelor's degree,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Graduate or professional degree
1,8600000US00601,601,ZCTA5 00601,17800,14436,108,21,0,0,3046,...,(X),(X),(X),(X),16533,11522,15606,13561,22859,36771
2,8600000US00602,602,ZCTA5 00602,39716,22941,968,0,74,0,1544,...,(X),(X),(X),(X),13682,8018,12037,13060,21037,26118
3,8600000US00603,603,ZCTA5 00603,51565,35176,2149,106,638,11,6610,...,(X),(X),(X),(X),16022,9848,11884,15023,24438,32807
4,8600000US00606,606,ZCTA5 00606,6320,3739,175,0,0,0,2246,...,(X),(X),(X),(X),10861,6116,12798,13690,14466,-
5,8600000US00610,610,ZCTA5 00610,27976,16259,1018,0,33,1,6871,...,(X),(X),(X),(X),15167,8367,14411,14419,21234,33125


In [34]:
census_df['zipcode_5'] = census_df['Id'].str[9:14]
new_census_df = census_df.iloc[:, 3:]
new_census_df.head()

,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,Estimate; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two races including Some other race,"Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races",...,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - High school graduate (includes equivalency),Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Some college or associate's degree,Total; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Bachelor's degree or higher,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Less than high school graduate,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - High school graduate (includes equivalency),Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Some college or associate's degree,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Bachelor's degree,Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Graduate or professional degree,zipcode_5
1,17800,14436,108,21,0,0,3046,189,26,163,...,(X),(X),(X),16533,11522,15606,13561,22859,36771,00601
2,39716,22941,968,0,74,0,1544,14189,164,14025,...,(X),(X),(X),13682,8018,12037,13060,21037,26118,00602
3,51565,35176,2149,106,638,11,6610,6875,333,6542,...,(X),(X),(X),16022,9848,11884,15023,24438,32807,00603
4,6320,3739,175,0,0,0,2246,160,77,83,...,(X),(X),(X),10861,6116,12798,13690,14466,-,00606
5,27976,16259,1018,0,33,1,6871,3794,103,3691,...,(X),(X),(X),15167,8367,14411,14419,21234,33125,00610


In [35]:
for column in new_census_df.columns[:211]:
    new_census_df[column] = pd.to_numeric(new_census_df[column], errors='coerce')
new_census_df.dropna(inplace=True, axis=1)
new_census_df.head()

,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,Estimate; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two races including Some other race,"Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races",...,Total; Estimate; Some other race alone,Total; Estimate; Some other race alone - High school graduate or higher,Total; Estimate; Some other race alone - Bachelor's degree or higher,Total; Estimate; Two or more races,Total; Estimate; Two or more races - High school graduate or higher,Total; Estimate; Two or more races - Bachelor's degree or higher,Total; Estimate; Hispanic or Latino Origin,Total; Estimate; Hispanic or Latino Origin - High school graduate or higher,Total; Estimate; Hispanic or Latino Origin - Bachelor's degree or higher,zipcode_5
1,17800,14436,108,21,0,0,3046,189,26,163,...,1924,1275,436,114,92,11,11863,7488,2217,00601
2,39716,22941,968,0,74,0,1544,14189,164,14025,...,849,653,218,9677,6399,2131,25503,16080,5073,00602
3,51565,35176,2149,106,638,11,6610,6875,333,6542,...,4474,3221,1100,4725,3033,1006,34582,23936,7604,00603
4,6320,3739,175,0,0,0,2246,160,77,83,...,1479,770,128,131,44,7,4370,2382,426,00606
5,27976,16259,1018,0,33,1,6871,3794,103,3691,...,4327,2950,560,2446,1705,326,18758,12590,2901,00610


In [36]:
combined_summary_df = new_census_df.merge(zip_summary_df, how='inner')
combined_summary_df.head()

,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,Estimate; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two races including Some other race,"Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races",...,Total; Estimate; Two or more races,Total; Estimate; Two or more races - High school graduate or higher,Total; Estimate; Two or more races - Bachelor's degree or higher,Total; Estimate; Hispanic or Latino Origin,Total; Estimate; Hispanic or Latino Origin - High school graduate or higher,Total; Estimate; Hispanic or Latino Origin - Bachelor's degree or higher,zipcode_5,donations_sum,donations_median,donations_count
0,17800,14436,108,21,0,0,3046,189,26,163,...,114,92,11,11863,7488,2217,00601,250.0,50.0,5
1,39716,22941,968,0,74,0,1544,14189,164,14025,...,9677,6399,2131,25503,16080,5073,00602,10316.0,50.0,62
2,51565,35176,2149,106,638,11,6610,6875,333,6542,...,4725,3033,1006,34582,23936,7604,00603,7481.0,15.0,61
3,6320,3739,175,0,0,0,2246,160,77,83,...,131,44,7,4370,2382,426,00606,510.0,5.0,3
4,27976,16259,1018,0,33,1,6871,3794,103,3691,...,2446,1705,326,18758,12590,2901,00610,5558.0,110.0,42


In [37]:
new_census_df.columns

Index(['Estimate; Total:', 'Estimate; Total: - White alone',
       'Estimate; Total: - Black or African American alone',
       'Estimate; Total: - American Indian and Alaska Native alone',
       'Estimate; Total: - Asian alone',
       'Estimate; Total: - Native Hawaiian and Other Pacific Islander alone',
       'Estimate; Total: - Some other race alone',
       'Estimate; Total: - Two or more races:',
       'Estimate; Total: - Two or more races: - Two races including Some other race',
       'Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races',
       ...
       'Total; Estimate; Some other race alone',
       'Total; Estimate; Some other race alone - High school graduate or higher',
       'Total; Estimate; Some other race alone - Bachelor's degree or higher',
       'Total; Estimate; Two or more races',
       'Total; Estimate; Two or more races - High school graduate or higher',
       'Total; Estimate; Two or more races - Bache

In [38]:
combined_summary_df['donations_sum_quartile'] = pd.qcut(combined_summary_df['donations_sum'], q=[0, .25, .5, .75, 1.], labels=[4, 3, 2, 1])
combined_summary_df['donations_median_quartile'] = pd.qcut(combined_summary_df['donations_median'], q=[0, .25, .5, .75, 1.], labels=[4, 3, 2, 1])

In [39]:
combined_summary_df.sort_values(by=['donations_sum'])

,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone,Estimate; Total: - American Indian and Alaska Native alone,Estimate; Total: - Asian alone,Estimate; Total: - Native Hawaiian and Other Pacific Islander alone,Estimate; Total: - Some other race alone,Estimate; Total: - Two or more races:,Estimate; Total: - Two or more races: - Two races including Some other race,"Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races",...,Total; Estimate; Two or more races - Bachelor's degree or higher,Total; Estimate; Hispanic or Latino Origin,Total; Estimate; Hispanic or Latino Origin - High school graduate or higher,Total; Estimate; Hispanic or Latino Origin - Bachelor's degree or higher,zipcode_5,donations_sum,donations_median,donations_count,donations_sum_quartile,donations_median_quartile
1079,51,51,0,0,0,0,0,0,0,0,...,0,0,0,0,04267,1.0,1.0,1,4,4
11596,503,369,133,0,0,0,0,1,0,1,...,0,0,0,0,39062,1.0,1.0,1,4,4
23778,206,195,0,4,0,0,6,1,0,1,...,0,32,24,0,76852,1.0,1.0,1,4,4
26189,1900,1428,0,72,0,0,144,256,5,251,...,10,817,497,13,85618,2.0,2.0,1,4,4
3484,371,321,9,0,3,0,0,38,0,38,...,0,0,0,0,12998,2.0,2.0,1,4,4
22375,581,561,9,6,0,0,0,5,0,5,...,0,0,0,0,72564,2.0,1.0,2,4,4
28885,770,716,0,4,0,0,20,30,20,10,...,20,40,22,20,97492,2.0,1.0,2,4,4
7980,220,220,0,0,0,0,0,0,0,0,...,0,0,0,0,28350,3.0,3.0,1,4,4
3334,1482,1036,142,0,15,0,264,25,0,25,...,0,203,62,0,12759,3.0,3.0,1,4,4
26641,389,385,2,0,0,0,2,0,0,0,...,0,156,130,2,88353,4.0,2.0,2,4,4


In [40]:
combined_summary_df.to_csv('cleaned_combined_zipcodes.csv', index=False)

In [41]:
combined_summary_df = pd.read_csv('cleaned_combined_zipcodes.csv')

In [42]:
from numpy.random import seed
seed(1)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [43]:
combined_summary_df.dropna(inplace=True)
X = combined_summary_df[combined_summary_df.columns[:163]]
y = combined_summary_df['donations_sum_quartile']
print(X.shape, y.shape)

(29721, 163) (29721,)


In [44]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [45]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [46]:
# The output layer has 5 columns that are one-hot encoded
y_train.shape

(22290, 5)

In [47]:
y_test

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [48]:
# Create an empty sequential model
model = Sequential()

In [49]:
# Add the first layer where the input dimensions are the 561 columns of the training data
model.add(Dense(200, activation='relu', input_dim=X_train.shape[1]))

Instructions for updating:
Colocations handled automatically by placer.


In [50]:
# Add a second hidden layer
model.add(Dense(200, activation='relu'))

In [51]:
# Add output layer
model.add(Dense(y_train.shape[1], activation="softmax"))

In [52]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [53]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
# Use the training data to fit (train) the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=20,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 - 2s - loss: 0.8214 - acc: 0.6234
Epoch 2/20
 - 2s - loss: 0.7568 - acc: 0.6556
Epoch 3/20
 - 2s - loss: 0.7388 - acc: 0.6701
Epoch 4/20
 - 2s - loss: 0.7319 - acc: 0.6739
Epoch 5/20
 - 2s - loss: 0.7192 - acc: 0.6792
Epoch 6/20
 - 2s - loss: 0.7137 - acc: 0.6777
Epoch 7/20
 - 2s - loss: 0.7084 - acc: 0.6848
Epoch 8/20
 - 2s - loss: 0.7059 - acc: 0.6835
Epoch 9/20
 - 2s - loss: 0.7007 - acc: 0.6864
Epoch 10/20
 - 2s - loss: 0.6961 - acc: 0.6913
Epoch 11/20
 - 2s - loss: 0.6941 - acc: 0.6895
Epoch 12/20
 - 2s - loss: 0.6927 - acc: 0.6903
Epoch 13/20
 - 2s - loss: 0.6852 - acc: 0.6911
Epoch 14/20
 - 2s - loss: 0.6800 - acc: 0.6974
Epoch 15/20
 - 2s - loss: 0.6761 - acc: 0.7012
Epoch 16/20
 - 2s - loss: 0.6711 - acc: 0.7011
Epoch 17/20
 - 2s - loss: 0.6694 - acc: 0.7026
Epoch 18/20
 - 2s - loss: 0.6646 - acc: 0.7040
Epoch 19/20
 - 2s - loss: 0.6604 - acc: 0.7074
Epoch 20/20
 - 2s - loss: 0.6534 - acc: 0.7100


In [139]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 0.7713 - acc: 0.6699
Loss: 0.7712671339608125, Accuracy: 0.6698963642120361


In [89]:
test = np.expand_dims(X_test_scaled[0], axis=0)
test.shape

(1, 8)

In [58]:
print(f"Predicted class: {model.predict_classes(test)}")

Predicted class: [1]
